In [1]:
from graph_tool.all import *
import itertools

In [4]:
import json

with open('data/announcements.json', 'r') as file:
    json_data = json.load(file)


In [5]:
from graph_tool.all import Graph

# Initialize an undirected graph
g = Graph(directed=False)

# Property maps for artist names and co-exhibition counts
artist_names = g.new_vertex_property("string")
co_exhibition_count = g.new_edge_property("int")

# Mapping artist names to vertices, to avoid duplicates
artist_to_vertex = {}

for exhibition in json_data.values():
    artists = exhibition["artists"]
    
    # Create a vertex for each unique artist
    for artist in artists:
        if artist not in artist_to_vertex:
            v = g.add_vertex()
            artist_names[v] = artist
            artist_to_vertex[artist] = v

# Add edges between artists who have exhibited together
for exhibition in json_data.values():
    artists = exhibition["artists"]
    
    # For each pair of artists, add an edge or update the count
    for i in range(len(artists)):
        for j in range(i+1, len(artists)):
            artist1 = artists[i]
            artist2 = artists[j]
            
            v1 = artist_to_vertex[artist1]
            v2 = artist_to_vertex[artist2]
            
            # Check if an edge already exists
            edge = g.edge(v1, v2, add_missing=False)
            if edge is None:
                # Add new edge with co-exhibition count = 1
                edge = g.add_edge(v1, v2)
                co_exhibition_count[edge] = 1
            else:
                # Increase co-exhibition count
                co_exhibition_count[edge] += 1

# Assign the property maps to the graph
g.vertex_properties["artist_names"] = artist_names
g.edge_properties["co_exhibition_count"] = co_exhibition_count

# Now g is your graph, ready for analysis and visualization.

In [8]:
from graph_tool.all import graph_draw, label_largest_component, betweenness, GraphView


# Basic analysis example: Print the number of vertices and edges
print(f"Number of artists (vertices): {g.num_vertices()}")
print(f"Number of co-exhibitions (edges): {g.num_edges()}")

# Identify and label the largest component (for visualization)
largest_component_mask = label_largest_component(g)
largest_component = GraphView(g, vfilt=largest_component_mask)

# Calculate betweenness centrality (as an example of centrality measures)
vertex_betweenness, edge_betweenness = betweenness(g)

# Visualization
# Adjust the vertex/edge sizes and colors based on properties such as centrality
graph_draw(g, vertex_fill_color=vertex_betweenness, edge_pen_width=edge_betweenness,
           output_size=(1000, 1000), output="co-exhibition_network.pdf")


Number of artists (vertices): 20938
Number of co-exhibitions (edges): 1149352


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x162f5de50, at 0x15625bc80>

There is one large component. Let's visualize only the largest component.

In [9]:
from graph_tool.all import graph_draw, GraphView, label_largest_component

# Assuming 'g' is your graph with all nodes and edges already added

# Create a filtered view of g that contains only the largest component
largest_component_filter = label_largest_component(g)
largest_component = GraphView(g, vfilt=largest_component_filter)

# Now, visualize the largest component
# You can adjust visualization parameters according to your needs
graph_draw(largest_component,
           vertex_text=largest_component.vertex_properties["artist_names"],
           vertex_font_size=10,
           output_size=(1000, 1000),
           output="largest_component_visualization.pdf")


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x167c82000, at 0x167c82b40>

The names are making the nodes too big. Let's get rid of them

In [12]:
from graph_tool.all import graph_draw, GraphView, label_largest_component

# Assuming 'g' is your graph with all nodes and edges already added

# Create a filtered view of g that contains only the largest component
largest_component_filter = label_largest_component(g)
largest_component = GraphView(g, vfilt=largest_component_filter)

# Visualize the largest component without labels and with smaller nodes
graph_draw(largest_component,
           vertex_size=10,  # Adjust vertex size here for smaller nodes
           output_size=(1000, 1000),
           output="largest_component_visualization_no_labels.pdf")


<VertexPropertyMap object with value type 'vector<double>', for Graph 0x167c82d20, at 0x167c81f40>